Load expressions
----------------

### imports ###

A hack to use local version o yspecies for debugging

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_bootstraps = 5 # this sets global setting of which how many bootstraps to use

lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'max_leaves': 20,
    'max_depth': 3,
    'learning_rate': 0.07,
    'feature_fraction': 0.8,
    'bagging_fraction': 1,
    'min_data_in_leaf': 6,
    'lambda_l1': 0.9,
    'lambda_l2': 0.9,
    "verbose": -1
}

debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from typing import *
from yspecies.enums import *
from yspecies.dataset import *
from yspecies.misc import *
from yspecies.workflow import *
from yspecies.partition import *
from yspecies.selection import *

In [4]:
from dataclasses import dataclass
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import shap
from pprint import pprint
import random
import numpy as np
import lightgbm as lgb
from scipy.stats import kendalltau
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.utils import resample
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, recall_score, precision_score, f1_score

In [6]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

### Loading data ###

In [7]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [8]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(452, 12630)",12630,43,452,"(12630, 2)","(44, 18)"


In [9]:
from sklearn.pipeline import Pipeline
selection = SelectedFeatures([],[], to_predict = "lifespan")
pipe = Pipeline([
    ('extractor', DataExtractor(selection)), 
    ("partitioner", DataPartitioner(species_in_validation=2)), 
    ("shap_computation", FeatureAnalyzer(ModelFactory()))]
)

In [10]:
d = pipe.fit_transform(data)
d

Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[149]	valid_0's l2: 16.8158	valid_0's l1: 2.24008


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[253]	valid_0's l2: 9.48604	valid_0's l1: 1.82453


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[257]	valid_0's l2: 17.658	valid_0's l1: 2.2897


...

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


Training until validation scores don't improve for 7 rounds
Early stopping, best iteration is:
[42]	valid_0's l2: 1110.14	valid_0's l1: 16.6857


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


MEAN metrics = R^2    -0.857
MSE   234.713
MAE     5.144
dtype: float64


FeatureResults(weights=                   symbol  gain_score_to_lifespan    shap  \
ensembl_id                                                  
ENSG00000175854      SWI5              281239.051  69.236   
ENSG00000060762      MPC1              102360.804 -68.189   
ENSG00000167515  TRAPPC2L              100906.652 -84.298   
ENSG00000066923     STAG3               44965.835  13.996   
ENSG00000010219     DYRK4               44661.724  -8.033   
ENSG00000129187      DCTD                4839.301   6.775   
ENSG00000066926      FECH                3622.228  -4.593   
ENSG00000165555   NOXRED1                2370.841   3.470   
ENSG00000213139     CRYGS                1692.762   3.788   
ENSG00000165689     ENTR1                1492.158   2.017   
ENSG00000116489    CAPZA1                1026.022  -7.030   
ENSG00000165501      LRR1                 958.245   2.527   
ENSG00000118600    RXYLT1                 716.958  -2.235   

                 kendall_tau_to_lifespan  
ensembl_id                                
ENSG00000175854                   -0.675  
ENSG00000060762                   -0.413  
ENSG00000167515                   -0.173  
ENSG00000066923                    0.532  
ENSG00000010219                    0.411  
ENSG00000129187                    0.685  
ENSG00000066926                   -0.527  
ENSG00000165555                    0.822  
ENSG00000213139                    0.615  
ENSG00000165689                   -0.666  
ENSG00000116489                   -0.083  
ENSG00000165501                    0.679  
ENSG00000118600                   -0.527  , stable_shap_values=array([[ 0.        ,  0.        , -0.00491749, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -0.00435626, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -0.00491749, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        , -0.00260712, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        , -0.00260712, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.00071846, ...,  0.        ,
         0.        ,  0.        ]]), metrics=     R^2      MSE    MAE
0  0.926   16.816  2.240
1  0.958    9.486  1.825
2  0.919   17.658  2.290
3  0.917   19.462  2.678
4 -8.004 1110.142 16.686)